In [1]:
import numpy as np
import os
import pickle
import string

import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk

In [2]:
from nltk.chunk import conlltags2tree, tree2conlltags

In [3]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm, en_core_web_lg


In [4]:
from nltk.corpus import wordnet as wn

In [5]:
from bs4 import BeautifulSoup
import requests
import re

In [6]:
nlp = en_core_web_lg.load()

In [6]:
with open('./unique_mpd_pl_title_clean_set.txt', 'r') as f:
    unique_mpd_pl_title_clean_list = f.readlines() 

with open('./descriptions_list.txt', 'r') as f:
    descriptions_list = f.readlines() 


In [9]:
# nltk.download('averaged_perceptron_tagger')
# nltk.download('maxent_ne_chunker')
# nltk.download('words')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/kaka/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [7]:
len(unique_mpd_pl_title_clean_list), len(descriptions_list)

(91584, 18761)

In [8]:
sentence = descriptions_list[1000]
ne_tree = ne_chunk(pos_tag(word_tokenize(sentence)))

In [9]:
print(ne_tree)

(S
  (GPE Finn/NNP)
  and/CC
  (PERSON Ben/NNP)
  like/IN
  this/DT
  you/PRP
  may/MD
  not/RB
  ./.
  There/EX
  you/PRP
  go/VBP
  ./.
  Have/VBP
  fun/NN
  ./.
  Follow/IN
  me/PRP
  please/NN
  ./.)


In [10]:
def preprocess(sent):
    sent = nltk.word_tokenize(sent)
    sent = nltk.pos_tag(sent)
    return sent

In [12]:
iob_tagged = tree2conlltags(ne_tree)

In [131]:
print(tree2conlltags(ne_chunk(pos_tag(word_tokenize(sentence)))))

[('Finn', 'NNP', 'B-GPE'), ('and', 'CC', 'O'), ('Ben', 'NNP', 'B-PERSON'), ('like', 'IN', 'O'), ('this', 'DT', 'O'), ('you', 'PRP', 'O'), ('may', 'MD', 'O'), ('not', 'RB', 'O'), ('.', '.', 'O'), ('There', 'EX', 'O'), ('you', 'PRP', 'O'), ('go', 'VBP', 'O'), ('.', '.', 'O'), ('Have', 'VBP', 'O'), ('fun', 'NN', 'O'), ('.', '.', 'O'), ('Follow', 'IN', 'O'), ('me', 'PRP', 'O'), ('please', 'NN', 'O'), ('.', '.', 'O')]


In [24]:
displacy.render(nlp(str(sentence)), style='dep', jupyter = True, options = {'distance': 80})

In [25]:
displacy.render(nlp(str(sentence)), jupyter=True, style='ent')

In [138]:
def remove_punct(text):
#     text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', ' ', text)
    text = re.sub('\W+', ' ', text)
    return re.split('\W+', text) 
#     return text

In [139]:
stopword = nltk.corpus.stopwords.words('english')
def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text

In [140]:
ps = nltk.PorterStemmer()
def stemming(text):
    text = [ps.stem(word) for word in text]
    return text

In [141]:
wn_lm = nltk.WordNetLemmatizer()
def lemmatizer(text):
    text = [wn_lm.lemmatize(word) for word in text]
    return text

In [142]:
def clean_text(text):
    return " ".join(lemmatizer(remove_stopwords(remove_punct(text))))

In [143]:
remove_punct("soccer/workout")

['soccer', 'workout']

In [145]:
for _desc in descriptions_list[3300:3310]:
    print('\n')
    print(_desc)
    print(clean_text(str(_desc)))
    displacy.render(nlp(clean_text(str(_desc))), style='ent', jupyter = True, options = {'distance': 80})
    print(tree2conlltags(ne_chunk(pos_tag(word_tokenize(clean_text(str(_desc)))))))
    for _word in clean_text(str(_desc)):
        possible_synsets = wn.synsets(_word,'n') # or 'a' or 'v'
        print(possible_synsets)



For those good days.....

For good day 


[('For', 'IN', 'O'), ('good', 'JJ', 'O'), ('day', 'NN', 'O')]
[Synset('degree_fahrenheit.n.01'), Synset('fluorine.n.01'), Synset('farad.n.01'), Synset('f.n.04')]
[Synset('oxygen.n.01'), Synset('o.n.02'), Synset('o.n.03')]
[Synset('roentgen.n.01'), Synset('gas_constant.n.01'), Synset('r.n.03'), Synset('radius.n.01')]
[]
[Synset('gram.n.01'), Synset('guanine.n.01'), Synset('deoxyguanosine_monophosphate.n.01'), Synset('thousand.n.01'), Synset('g.n.05'), Synset('gigabyte.n.01'), Synset('gigabyte.n.02'), Synset('gravitational_constant.n.01'), Synset('g.n.09')]
[Synset('oxygen.n.01'), Synset('o.n.02'), Synset('o.n.03')]
[Synset('oxygen.n.01'), Synset('o.n.02'), Synset('o.n.03')]
[Synset('vitamin_d.n.01'), Synset('five_hundred.n.01'), Synset('d.n.03')]
[]
[Synset('vitamin_d.n.01'), Synset('five_hundred.n.01'), Synset('d.n.03')]
[Synset('angstrom.n.01'), Synset('vitamin_a.n.01'), Synset('deoxyadenosine_monophosphate.n.01'), Synset('adenine.n.01'), Synset('ampere.n.02'), Synset('a.n.06'), Synse

/usr/lib/python3.5/runpy.py:184: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


[('Heres', 'NNS', 'O'), ('change', 'VBP', 'O'), ('pace', 'NN', 'O'), ('change', 'NN', 'O'), ('vibe', 'NN', 'O')]
[Synset('hydrogen.n.01'), Synset('henry.n.01'), Synset('planck's_constant.n.01'), Synset('h.n.04'), Synset('heat_content.n.01')]
[Synset('vitamin_e.n.01'), Synset('einsteinium.n.01'), Synset('east.n.01'), Synset('e.n.04'), Synset('e.n.05')]
[Synset('roentgen.n.01'), Synset('gas_constant.n.01'), Synset('r.n.03'), Synset('radius.n.01')]
[Synset('vitamin_e.n.01'), Synset('einsteinium.n.01'), Synset('east.n.01'), Synset('e.n.04'), Synset('e.n.05')]
[Synset('second.n.01'), Synset('sulfur.n.01'), Synset('south.n.03'), Synset('mho.n.01'), Synset('s.n.05'), Synset('randomness.n.01')]
[]
[Synset('degree_centigrade.n.01'), Synset('speed_of_light.n.01'), Synset('vitamin_c.n.01'), Synset('deoxycytidine_monophosphate.n.01'), Synset('cytosine.n.01'), Synset('carbon.n.01'), Synset('hundred.n.01'), Synset('coulomb.n.01'), Synset('c.n.09'), Synset('c.n.10'), Synset('c.n.11'), Synset('coke.n.

/usr/lib/python3.5/runpy.py:184: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


...

/usr/lib/python3.5/runpy.py:184: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


[('love', 'NN', 'O'), ('britt', 'NN', 'O')]
[Synset('liter.n.01'), Synset('fifty.n.01'), Synset('lambert.n.01'), Synset('l.n.04')]
[Synset('oxygen.n.01'), Synset('o.n.02'), Synset('o.n.03')]
[Synset('volt.n.01'), Synset('vanadium.n.01'), Synset('five.n.01'), Synset('v.n.04')]
[Synset('vitamin_e.n.01'), Synset('einsteinium.n.01'), Synset('east.n.01'), Synset('e.n.04'), Synset('e.n.05')]
[]
[Synset('bacillus.n.01'), Synset('b-complex_vitamin.n.01'), Synset('boron.n.01'), Synset('bel.n.01'), Synset('barn.n.02'), Synset('b.n.06'), Synset('b.n.07')]
[Synset('roentgen.n.01'), Synset('gas_constant.n.01'), Synset('r.n.03'), Synset('radius.n.01')]
[Synset('iodine.n.01'), Synset('one.n.01'), Synset('i.n.03')]
[Synset('thymine.n.01'), Synset('deoxythymidine_monophosphate.n.01'), Synset('metric_ton.n.01'), Synset('t.n.04'), Synset('triiodothyronine.n.01'), Synset('thyroxine.n.01')]
[Synset('thymine.n.01'), Synset('deoxythymidine_monophosphate.n.01'), Synset('metric_ton.n.01'), Synset('t.n.04'), Sy

[('When', 'WRB', 'O'), ('piano', 'NN', 'O'), ('fore', 'RB', 'O'), ('music', 'NN', 'O'), ('often', 'RB', 'O'), ('swing', 'VBG', 'O'), ('sometimes', 'RB', 'O'), ('surprise', 'VB', 'O'), ('coolest', 'JJS', 'O'), ('sound', 'NN', 'O'), ('around', 'IN', 'O'), ('Featuring', 'NNP', 'O'), ('Herbie', 'NNP', 'B-PERSON'), ('Hancock', 'NNP', 'I-PERSON'), ('Bill', 'NNP', 'B-PERSON'), ('Evans', 'NNP', 'I-PERSON'), ('Oscar', 'NNP', 'I-PERSON'), ('Peterson', 'NNP', 'I-PERSON'), ('Bud', 'NNP', 'I-PERSON'), ('Powell', 'NNP', 'I-PERSON'), ('Teddy', 'NNP', 'I-PERSON'), ('Wilson', 'NNP', 'I-PERSON'), ('many', 'JJ', 'O')]
[Synset('tungsten.n.01'), Synset('west.n.02'), Synset('watt.n.01'), Synset('w.n.04')]
[Synset('hydrogen.n.01'), Synset('henry.n.01'), Synset('planck's_constant.n.01'), Synset('h.n.04'), Synset('heat_content.n.01')]
[Synset('vitamin_e.n.01'), Synset('einsteinium.n.01'), Synset('east.n.01'), Synset('e.n.04'), Synset('e.n.05')]
[Synset('nitrogen.n.01'), Synset('north.n.03'), Synset('newton.n.0

/usr/lib/python3.5/runpy.py:184: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


[]
[]
